In [1]:
!pwd && ls -a
# Install additional libs
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/trl.git@7630f877f91c556d9e5a3baa4b6e2894d90ff84c
!pip install ua_gec
!pip install datasets==2.16.0
!pip install nltk
!pip install wandb -q -U

/kaggle/working
.  ..  .virtual_documents
  Cloning https://github.com/huggingface/trl.git (to revision 7630f877f91c556d9e5a3baa4b6e2894d90ff84c) to /tmp/pip-req-build-ecwjjc3t
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-ecwjjc3t
  Running command git rev-parse -q --verify 'sha^7630f877f91c556d9e5a3baa4b6e2894d90ff84c'
  Running command git fetch -q https://github.com/huggingface/trl.git 7630f877f91c556d9e5a3baa4b6e2894d90ff84c
  Running command git checkout -q 7630f877f91c556d9e5a3baa4b6e2894d90ff84c
  Resolved https://github.com/huggingface/trl.git to commit 7630f877f91c556d9e5a3baa4b6e2894d90ff84c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 3.0 MB/s eta 0:00:00
  Created wheel for trl: filename=trl-0.7.12.dev0-py3-none-any.whl size=173433 sha256=9c140761e84804829814e749

In [2]:
from transformers import AutoModelForCausalLM, pipeline, Conversation, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, TaskType, PeftModel, get_peft_model, prepare_model_for_kbit_training
from kaggle_secrets import UserSecretsClient
import torch
import nltk
import wandb

nltk.download('punkt')  # Download the necessary resources for sentence tokenization

from nltk.tokenize import sent_tokenize

2024-03-24 22:24:30.599843: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 22:24:30.599938: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 22:24:30.748270: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
import os
os.environ["PYTORCH_USE_CUDA_DSA"] = "0"

Load HuggingFace and Weights & Biases secrets

In [5]:
user_secrets = UserSecretsClient()
secret_hf = user_secrets.get_secret("HUGGINGFACE_TOKEN")
secret_wandb = user_secrets.get_secret("wandb")

Login to HuggingFace

In [6]:
!huggingface-cli login --token $secret_hf

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
fine_tuned_model_name = 'rkovalchuk/mistral-7b-ua-gec'
merged_model_name = "mistral-7b-it-ua-gec-merged"

In [8]:
lora_adapter = fine_tuned_model_name
base_model = model_name

model_to_merge = PeftModel.from_pretrained(
    AutoModelForCausalLM.from_pretrained(
        base_model,
        torch_dtype=torch.bfloat16,
    ).to("cuda"),
    lora_adapter
)

merged_model = model_to_merge.merge_and_unload()
merged_model.save_pretrained(merged_model_name)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/23.1M [00:00<?, ?B/s]

In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left', trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [10]:
output_dir = "/kaggle/working/mistral-7b-it-ua-gec-merged"
tokenizer.save_pretrained(output_dir, legacy_format=False)

('/kaggle/working/mistral-7b-it-ua-gec-merged/tokenizer_config.json',
 '/kaggle/working/mistral-7b-it-ua-gec-merged/special_tokens_map.json',
 '/kaggle/working/mistral-7b-it-ua-gec-merged/tokenizer.json')

In [11]:
%cd /kaggle/working

/kaggle/working


In [15]:
# !git clone https://github.com/ggerganov/llama.cpp
# %cd llama.cpp/ 
# !python3 -m pip install -r requirements.txt
# !site_packages=$(python -c "from distutils.sysconfig import get_python_lib; print(get_python_lib())")
# !rm -rf "$site_packages"/numpy
# !pip install --upgrade --force-reinstall numpy
!python3 convert.py ../mistral-7b-it-ua-gec-merged/ --outtype q8_0

Loading model file ../mistral-7b-it-ua-gec-merged/model-00001-of-00003.safetensors
Loading model file ../mistral-7b-it-ua-gec-merged/model-00001-of-00003.safetensors
Loading model file ../mistral-7b-it-ua-gec-merged/model-00002-of-00003.safetensors
Loading model file ../mistral-7b-it-ua-gec-merged/model-00003-of-00003.safetensors
params = Params(n_vocab=32000, n_embd=4096, n_layer=32, n_ctx=32768, n_ff=14336, n_head=32, n_head_kv=8, n_experts=None, n_experts_used=None, f_norm_eps=1e-05, rope_scaling_type=None, f_rope_freq_base=1000000.0, f_rope_scale=None, n_orig_ctx=None, rope_finetuned=None, ftype=<GGMLFileType.MostlyQ8_0: 7>, path_model=PosixPath('../mistral-7b-it-ua-gec-merged'))
Found vocab files: {'spm': None, 'bpe': None, 'hfft': PosixPath('../mistral-7b-it-ua-gec-merged/tokenizer.json')}
Loading vocab file PosixPath('../mistral-7b-it-ua-gec-merged/tokenizer.json'), type 'hfft'
fname_tokenizer: ../mistral-7b-it-ua-gec-merged
Vocab info: <HfVocab with 32000 base tokens and 0 adde

In [16]:
merged_model.push_to_hub(merged_model_name)

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rkovalchuk/mistral-7b-it-ua-gec-merged/commit/5cd8f15772ee161e9433135f41909d3573a13b0d', commit_message='Upload MistralForCausalLM', commit_description='', oid='5cd8f15772ee161e9433135f41909d3573a13b0d', pr_url=None, pr_revision=None, pr_num=None)